# In-Class Exercise Q2: Longest Common Subsequence (DNA)

### **Goal**: Compute the LCS for a = ABCBDAB and b = BDCABA using dynamic programming; show the filled table (optionally) and backtrack one LCS.

# 1) Setup & Functions

In [1]:
# Sequences
a = "ABCBDAB"
b = "BDCABA"


# DP build: lengths table (dp) + move arrows (↖ match, ↑ from top, ← from left)


def lcs_tables(a: str, b: str):
    m, n = len(a), len(b)
    dp = [[0]*(n+1) for _ in range(m+1)]
    arrows = [[""]*(n+1) for _ in range(m+1)]
    for i in range(1, m+1):
        for j in range(1, n+1):
            if a[i-1] == b[j-1]:
                dp[i][j] = dp[i-1][j-1] + 1
                arrows[i][j] = "↖"
            else:
                if dp[i-1][j] >= dp[i][j-1]: # prefer up on ties
                    dp[i][j] = dp[i-1][j]
                    arrows[i][j] = "↑"
                else:
                    dp[i][j] = dp[i][j-1]
                    arrows[i][j] = "←"
    return dp, arrows


# Backtrack one LCS by following arrows from bottom‑right


def backtrack_lcs(a: str, b: str, arrows):
    i, j = len(a), len(b)
    out = []
    while i > 0 and j > 0:
        move = arrows[i][j]
        if move == "↖":
            out.append(a[i-1])
            i -= 1
            j -= 1
        elif move == "↑":
            i -= 1
        else: # "←" or ""
            j -= 1
    return "".join(reversed(out))

# 2) Run & Report

In [2]:
# Build tables
dp, arrows = lcs_tables(a, b)


# LCS length and one valid LCS
lcs_one = backtrack_lcs(a, b, arrows)
length = dp[len(a)][len(b)]


print(f"a = {a}\nb = {b}")
print(f"LCS length = {length}")
print(f"One LCS = {lcs_one}")

a = ABCBDAB
b = BDCABA
LCS length = 4
One LCS = BCBA


#3) Tables

In [3]:
import pandas as pd

m, n = len(a), len(b)

# Build labeled DP table
header = ["i/j", "∅"] + list(b)
rows_dp = []
for i in range(m+1):
    label = "∅" if i == 0 else a[i-1]
    rows_dp.append([label] + [dp[i][j] for j in range(n+1)])

# Build labeled arrow table
rows_ar = []
for i in range(m+1):
    label = "∅" if i == 0 else a[i-1]
    rows_ar.append([label] + [arrows[i][j] if arrows[i][j] else "" for j in range(n+1)])

if pd is not None:
    display(pd.DataFrame(rows_dp, columns=header))
    display(pd.DataFrame(rows_ar, columns=header))
else:
    print("\nDP (lengths) table:")
    print("\t".join(map(str, header)))
    for r in rows_dp:
        print("\t".join(map(str, r)))
    print("\nArrow table (↖/↑/←):")
    print("\t".join(map(str, header)))
    for r in rows_ar:
        print("\t".join(map(str, r)))

,i/j,∅,B,D,C,A,B,A
0,∅,0,0,0,0,0,0,0
1,A,0,0,0,0,1,1,1
2,B,0,1,1,1,1,2,2
3,C,0,1,1,2,2,2,2
4,B,0,1,1,2,2,3,3
5,D,0,1,2,2,2,3,3
6,A,0,1,2,2,3,3,4
7,B,0,1,2,2,3,4,4


,i/j,∅,B,D,C,A,B,A
0,∅,,,,,,,
1,A,,↑,↑,↑,↖,←,↖
2,B,,↖,←,←,↑,↖,←
3,C,,↑,↑,↖,←,↑,↑
4,B,,↖,↑,↑,↑,↖,←
5,D,,↑,↖,↑,↑,↑,↑
6,A,,↑,↑,↑,↖,↑,↖
7,B,,↖,↑,↑,↑,↖,↑


# 4) Notes and Summary

Sequences:

*   a = ABCBDAB
*   b = BDCABA

DP Table Observations:

*   The DP table shows how the LCS length builds up as we compare prefixes of a and b.
*   Each cell dp\[i][j] contains the length of the longest common subsequence between a\[:i] and b\[:j].
*   The arrow table shows which direction the value came from:
    *   ↖ means characters matched and contributed to the subsequence.
    *   ↑ means we copied from the row above (skipped a character from a).
    *   ← means we copied from the column to the left (skipped a character from b).

Result from DP Table:

*   Bottom-right cell = 4, so the LCS length is 4.
*   Backtracking through the arrows recovers one valid LCS.
*   One LCS found: BCBA.

Interpretation of Backtracking:

*   We start at the bottom-right corner (i=7, j=6).
*   Follow the arrows:
    *   When we hit a diagonal (↖), we add the matching letter to the subsequence.
    *   When we hit ↑ or ←, we skip one character from the corresponding sequence.
*   This process reconstructs the subsequence in reverse, which we then reverse at the end to get the LCS.

Summary:

*   The longest common subsequence of ABCBDAB and BDCABA has length 4.
*   One valid LCS is BCBA.
*   There can be multiple equally long LCS solutions (e.g., BDAB, BCAB), but they all share the same maximum length.